# Final Project #
#### CSCI 4831 - Sabermetrics ####
#### Instructor: Rhonda Hoenigman ####
#### Name: Hussain Quadri ####

In [1]:
#Import Statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pybaseball import pitching_stats_range, pitching_stats_bref
from pybaseball import statcast
%matplotlib inline


## Data & Statistics

In [ ]:
# Data grabbing
# data = statcast('2017-01-01', '2018-12-31')
pitchers_2017 = pd.read_csv("Statcast_Pitching_Data_2017.csv", low_memory=False)
pitchers_2018 = pd.read_csv("Statcast_Pitching_Data_2018.csv", low_memory=False)
pitchers = pd.concat([pitchers_2017, pitchers_2018])

In [ ]:
pitchers.head(5)

In [ ]:
#Choosing variables

d = pitchers.copy()
fig, ax1 = plt.subplots()
ax1.set_xlabel("Launch Angle")
ax1.set_ylabel("Launch Speed")
ax1.set_title("Relation of Launch Angle vs Launch Speed")
ax1.scatter(d.launch_angle, d.launch_speed, s=2)


It is interesting to see here how fast a ball is thrown depending on the angle it is thrown at, the sweet spot being anywhere from 0-50 degrees, suggesting fast straight/curve balls. This makes me think that these variables have an importance in pitching quality.

In [ ]:
# Filter data for just bb_type, v(x,y,z)0/a(x,y,z) effective_speed, release_spin, launch_speed, and launch_speed_angle

filtered_data = pitchers[['player_name',
                 'pitcher',
                 'bb_type',
                 'vx0', 'vy0', 'vz0',
                 'ax', 'ay', 'az',
                 'effective_speed',
                 'release_spin_rate',
                 'launch_speed',
                 'launch_speed_angle', 'woba_value']].groupby('player_name', as_index = False).mean()
filtered_data.head(10)

In [ ]:
#Finding variance in variables to figure out weighting system

filtered_data.var()



In this, we can see the most variance is in the release_spin_rate, and the least in launch_speed_angle
Weighting will be done on a scale of 1-10, with 1 going to the variable with the highest variance, and 10 going to the variable with the lowest variance

In [ ]:
# Weighted Statistic = PitchingQuality = ((10*launch_speed_angle + 9*vz0 + 8*ay + 7*effective_speed + 6*az
#                                            + 5*vy0 + 4*launch_speed + 3*vx0 + 2*ax + release_spin_rate)/(55*100))

filtered_data['Pitching_Quality'] = ((10*filtered_data['launch_speed_angle'] + 9*filtered_data['vz0'] + 8*filtered_data['ay'] 
                                      + 7*filtered_data['effective_speed'] + 6*filtered_data['az'] + 5*filtered_data['vy0'] 
                                      + 4*filtered_data['launch_speed'] + 3*filtered_data['vx0'] + 2*filtered_data['ax'] 
                                      + filtered_data['release_spin_rate'])/(27.5*100))

Updated_Data = filtered_data.sort_values(by=['Pitching_Quality'], ascending=False)
Updated_Data.head()

In [ ]:
Updated_Data.head(20).plot.bar(x='player_name', y='Pitching_Quality', figsize=(10, 10))

The plot above shows a sample of the top 20 players according to this statistic. Now let's compare it to the top 20 pitchers using wOBA as an estimate.

In [ ]:
filtered_data.sort_values(by=['woba_value'], ascending=False).head(20).plot.bar(x='player_name', y='woba_value', figsize=(10, 10))

The wOBA statistic shows the offensive contributions of a player on a plate appearance, but I would like to see how offensiveness relates to the pitching quality.

In [ ]:
filtered_data.sort_values(by=['woba_value'], ascending=False).head(20).plot.bar(x='player_name', y=['woba_value', 'Pitching_Quality'], figsize=(10, 10), stacked=True)

Generally, it seems as if wOBA has a slight correlation to pitching quality, i.e the figure above is showing that a generally high wOBA score most likely relates to a high pitch quality.